In [1]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt 
import autoemulate as ae
from tqdm import tqdm

from autoemulate.experimental_design import LatinHypercube
from autoemulate.simulations.reaction_diffusion import simulate_reaction_diffusion
from autoemulate.compare import AutoEmulate
from autoemulate.plotting import _predict_with_optional_std

### 1) Data generation

In [2]:
from autoemulate.experimental.data.the_well import AutoEmulateWellDatasetTabular

data = AutoEmulateWellDatasetTabular(
    field_indices=[0],
    well_base_path="../../notebooks/the_well/datasets/",
    well_dataset_name="active_matter",
    well_split_name="test",
)
x, y = data.get_data()

In [3]:
def plot_simulation(idx):
    import matplotlib.animation as animation
    fig, ax = plt.subplots()
    im = ax.imshow(y[idx, 0].reshape(256, 256), animated=True)
    def update(frame):
        im.set_array(y[idx, frame].reshape(256, 256))
        ax.set_title(f"Frame {frame}")
        return [im]

    ani = animation.FuncAnimation(fig, update, frames=y.shape[1], interval=100, blit=True)
    plt.close(fig)
    return ani

In [4]:
from IPython.display import HTML
ani = plot_simulation(20)
HTML(ani.to_jshtml())

## 2) Reduced-dimension Emulator

In [5]:
from sklearn.decomposition import PCA
from sklearn.model_selection import KFold

x, y = x.reshape(-1, x.shape[-1]), y.reshape(-1, y.shape[-1])
em = AutoEmulate()
cross_validator = KFold(
            n_splits=2, shuffle=True, random_state=np.random.randint(1e5)
        )

preprocessing_methods = [
    {"name": "PCA", "params": {"reduced_dim": 64}}
]

em.setup(x, y, models=["gp"], scale_output = False, cross_validator = cross_validator, reduce_dim_output=True, preprocessing_methods=preprocessing_methods)

best_model = em.compare()

,Values
Simulation input shape (X),"(2106, 4)"
Simulation output shape (y),"(2106, 65536)"
Proportion of data for testing (test_set_size),0.2
Scale input data (scale),True
Scaler (scaler),StandardScaler
Scale output data (scale_output),False
Do hyperparameter search (param_search),False
Reduce input dimensionality (reduce_dim),False
Reduce output dimensionality (reduce_dim_output),True
Dimensionality output reduction methods (dim_reducer_output),PCA


Cross-validating:   0%|          | 0/1 [00:00<?, ?it/s]

In [6]:
best_model

InputOutputPipeline(regressor=Pipeline(steps=[('scaler', StandardScaler()),
                                              ('model', GaussianProcess())]),
                    transformer=Pipeline(steps=[('dim_reducer_output',
                                                 PCA(n_components=64))]))

## 3) Summarising cross-validation results

In [7]:
em.summarise_cv()

,preprocessing,model,short,fold,rmse,r2
0,PCA,GaussianProcess,gp,0,0.002151,0.333534
1,PCA,GaussianProcess,gp,1,0.002212,0.332788


In [8]:
## 3) Evaluate the emulator (on the test set)
gp = em.get_model('GaussianProcess')
em.evaluate(gp)

,model,short,preprocessing,rmse,r2
0,GaussianProcess,gp,PCA,0.0021,0.4507


## 4) Refitting the model on the full dataset

In [9]:
# gp_final = em.refit(em.get_model())

## 5) Predict on the test set
